In [1]:
import os
import torch
import argparse
import itertools
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import get_rank, init_process_group, destroy_process_group, all_gather, get_world_size
from torch import Tensor
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from glob import glob
from torch.utils.data.distributed import DistributedSampler
import random
from conditionDiffusion.unet import Unet
from conditionDiffusion.embedding import ConditionalEmbedding
from conditionDiffusion.utils import get_named_beta_schedule
from conditionDiffusion.diffusion import GaussianDiffusion
from conditionDiffusion.Scheduler import GradualWarmupScheduler
from PIL import Image
import torchvision
print(f"GPUs used:\t{torch.cuda.device_count()}")
device = torch.device("cuda",3)
print(f"Device:\t\t{device}")
import pytorch_model_summary as tms

GPUs used:	8
Device:		cuda:3


In [2]:
class_list=['유형8','유형9']
params={'image_size':1024,
        'lr':2e-5,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':2,
        'epochs':1000,
        'n_classes':None,
        'data_path':'../../data/normalization_type/BRIL/',
        'image_count':5000,
        'inch':3,
        'modch':32,
        'outch':3,
        'chmul':[1,2,4,8,16,32,64],
        'numres':2,
        'dtype':torch.float32,
        'cdim':10,
        'useconv':False,
        'droprate':0.1,
        'T':1000,
        'w':1.8,
        'v':0.3,
        'multiplier':1,
        'threshold':0.1,
        'ddim':True,
        }


In [3]:
trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

def transback(data:Tensor) -> Tensor:
    return data / 2 + 0.5

class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self,parmas, images,label):
        
        self.images = images
        self.args=parmas
        self.label=label
        
    def trans(self,image):
        if random.random() > 0.5:
            transform = transforms.RandomHorizontalFlip(1)
            image = transform(image)
            
        if random.random() > 0.5:
            transform = transforms.RandomVerticalFlip(1)
            image = transform(image)
            
        return image
    
    def __getitem__(self, index):
        image=self.images[index]
        label=self.label[index]
        image = self.trans(image)
        return image,label
    
    def __len__(self):
        return len(self.images)


image_label=[]
image_path=[]
for i in tqdm(range(len(class_list))):
    image_list=glob(params['data_path']+class_list[i]+'/*.jpeg')
    for j in range(len(image_list)):
        image_path.append(image_list[j])
        image_label.append(i)
        
train_images=torch.zeros((len(image_path),params['inch'],params['image_size'],params['image_size']))
for i in tqdm(range(len(image_path))):
    train_images[i]=trans(Image.open(image_path[i]).convert('RGB').resize((params['image_size'],params['image_size'])))
train_dataset=CustomDataset(params,train_images,image_label)
dataloader=DataLoader(train_dataset,batch_size=params['batch_size'],shuffle=True)

100%|██████████| 916/916 [02:00<00:00,  7.60it/s]


In [4]:
net = Unet(in_ch = params['inch'],
            mod_ch = params['modch'],
            out_ch = params['outch'],
            ch_mul = params['chmul'],
            num_res_blocks = params['numres'],
            cdim = params['cdim'],
            use_conv = params['useconv'],
            droprate = params['droprate'],
            dtype = params['dtype']
            ).to(device)
cemblayer = ConditionalEmbedding(len(class_list), params['cdim'], params['cdim']).to(device)
betas = get_named_beta_schedule(num_diffusion_timesteps = params['T'])
diffusion = GaussianDiffusion(
                    dtype = params['dtype'],
                    model = net,
                    betas = betas,
                    w = params['w'],
                    v = params['v'],
                    device = device
                )
optimizer = torch.optim.AdamW(
                itertools.chain(
                    diffusion.model.parameters(),
                    cemblayer.parameters()
                ),
                lr = params['lr'],
                weight_decay = 1e-6
            )


cosineScheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
warmUpScheduler = GradualWarmupScheduler(
                        optimizer = optimizer,
                        multiplier = params['multiplier'],
                        warm_epoch = 10,
                        after_scheduler = cosineScheduler,
                        last_epoch = 0
                    )
# checkpoint=torch.load(f'../../model/conditionDiff/BR/ckpt_35_checkpoint.pt',map_location=device)
# diffusion.model.load_state_dict(checkpoint['net'])

checkpoint=0

In [5]:
scaler = torch.cuda.amp.GradScaler()
topilimage = torchvision.transforms.ToPILImage()
for epc in range(params['epochs']):
    diffusion.model.train()
    cemblayer.train()
    total_loss=0
    steps=0
    with tqdm(dataloader, dynamic_ncols=True) as tqdmDataLoader:
        for img, lab in tqdmDataLoader:
            b = img.shape[0]
            
            x_0 = img.to(device)
            lab = lab.to(device)
            cemb = cemblayer(lab)
            cemb[np.where(np.random.rand(b)<params['threshold'])] = 0
            loss = diffusion.trainloss(x_0, cemb = cemb)
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            steps+=1
            total_loss+=loss.item()
            tqdmDataLoader.set_postfix(
                ordered_dict={
                    "epoch": epc + 1,
                    "loss: ": total_loss/steps,
                    "batch per device: ":x_0.shape[0],
                    "img shape: ": x_0.shape[1:],
                    "LR": optimizer.state_dict()['param_groups'][0]["lr"]
                }
            )
    warmUpScheduler.step()
    
    diffusion.model.eval()
    cemblayer.eval()
    # generating samples
    # The model generate 80 pictures(8 per row) each time
    # pictures of same row belong to the same class
    all_samples = []
    each_device_batch =len(class_list)
    with torch.no_grad():
        lab = torch.ones(len(class_list), each_device_batch // len(class_list)).type(torch.long) \
        * torch.arange(start = 0, end = len(class_list)).reshape(-1, 1)
        lab = lab.reshape(-1, 1).squeeze()
        lab = lab.to(device)
        cemb = cemblayer(lab)
        genshape = (each_device_batch , 3, params['image_size'], params['image_size'])
        if params['ddim']:
            generated = diffusion.ddim_sample(
                genshape, 100, 0.5, 'quadratic', cemb=cemb)
        else:
            generated = diffusion.sample(genshape, cemb = cemb)
        generated=transback(generated)
        for i in range(len(lab)):
            img_pil = topilimage(generated[i].cpu())
            img_pil.save(f'../../result/Detail/BRIL/{class_list[lab[i]]}/{epc}.png')

        # save checkpoints
        checkpoint = {
                            'net':diffusion.model.state_dict(),
                            'cemblayer':cemblayer.state_dict(),
                            'optimizer':optimizer.state_dict(),
                            'scheduler':warmUpScheduler.state_dict()
                        }
    torch.save(checkpoint, f'../../model/conditionDiff/details/BRIL/ckpt_{epc+1}_checkpoint.pt')
    torch.cuda.empty_cache()
    

  0%|          | 0/458 [00:00<?, ?it/s]

100%|██████████| 458/458 [07:25<00:00,  1.03it/s, epoch=1, loss: =0.468, batch per device: =2, img shape: =torch.Size([3, 1024, 1024]), LR=2e-5]


Start generating(ddim)...


100%|██████████| 100/100 [00:45<00:00,  2.19it/s]


ending sampling process(ddim)...


...

Start generating(ddim)...


100%|██████████| 100/100 [00:45<00:00,  2.19it/s]


ending sampling process(ddim)...


 81%|████████  | 372/458 [05:57<01:22,  1.04it/s, epoch=41, loss: =0.0424, batch per device: =2, img shape: =torch.Size([3, 1024, 1024]), LR=4.52e-6]


KeyboardInterrupt: 